In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 64.1 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 6.6 MB 41.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip3 install future

In [ ]:
!pip3 install lshash3

     |████████████████████████████████| 223 kB 5.1 MB/s 
  Created wheel for lshash3: filename=lshash3-0.0.8-py2.py3-none-any.whl size=10164 sha256=d27d12d262f2b512420724e74be4ea3150f16bba3884f31c285107986d75e7f7
  Stored in directory: /root/.cache/pip/wheels/65/72/01/dd1b56064187746f99aa560bbc491048575621ce6bb37cd05a
Successfully built lshash3


In [ ]:
from google.colab import drive
import pandas
from lshash import lshash
import sys, json
import pickle
drive.mount("/content/gdrive")
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt')

Mounted at /content/gdrive


In [ ]:
from Selectra import SELECTRA, pass_through_selectra
from transformers import ElectraTokenizer, ElectraModel, ElectraConfig
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
model = SELECTRA(n_classes = 2)
model.load_state_dict(torch.load("/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt/LSH/best_model_state.bin",map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SELECTRA(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=256, out_featur

In [ ]:
MODEL_NAME = "google/electra-small-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt/LSH/sentences_dict.json', 'r') as fp:
    sent_dict = json.load(fp)
    k = 10 # hash size
    L = 5  # number of tables
    d = 256 # Dimension of Feature vector.
    # LSH on all the images
    lsh = lshash.LSHash(hash_size=k, input_dim=d, num_hashtables=L,
                   matrices_filename='weights.npz',
                   overwrite=True)
    for index,sentence in sent_dict.items():
      embedding = pass_through_selectra(sentence, tokenizer, model, device).flatten()
      lsh.index(embedding.to("cpu").detach().numpy(), extra_data=index)
      if int(index)%10000 == 0:
        print(index)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000


In [ ]:
question = "Can I marry my cousin?"
embedding = pass_through_selectra(question, tokenizer, model, device).flatten()
response = lsh.query(embedding.to("cpu").detach().numpy(), num_results=3,distance_func = "euclidean" )

In [ ]:
print(response[0][0][1])
print(response[1][0][1])
print(response[2][0][1])

361130
356143
48580


In [ ]:
print(sent_dict[response[0][0][1]])
print(sent_dict[response[1][0][1]])
print(sent_dict[response[2][0][1]])

Could I marry my cousin?
Is it difficult to hit on a girl who already has a boyfriend?
Could you marry/date a porn star, why or why not?


In [ ]:
file_to_store = open("/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt/LSH/lsh_model.pickle", "wb")
pickle.dump(lsh, file_to_store)
file_to_store.close()